In [383]:

import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

load DAATA


In [384]:
TICKER = "^VNINDEX.VN"          
PERIOD = "720d"               
INTERVAL = "60m"              
df = yf.download(TICKER, period=PERIOD, interval=INTERVAL, auto_adjust=True)
df.to_csv('Data_VNIDEX_1H.csv')


[*********************100%***********************]  1 of 1 completed


đọc data và hiểu

In [385]:
df = pd.read_csv('Data_VNIDEX_1H.csv')

In [386]:
df.head()


,Price,Close,High,Low,Open,Volume
0,Ticker,^VNINDEX.VN,^VNINDEX.VN,^VNINDEX.VN,^VNINDEX.VN,^VNINDEX.VN
1,Datetime,NaN,NaN,NaN,NaN,NaN
2,2023-07-20 02:00:00+00:00,1170.719970703125,1174.3900146484375,1169.7099609375,1172.97998046875,0
3,2023-07-20 03:00:00+00:00,1168.0400390625,1171.3199462890625,1167.6800537109375,1170.6600341796875,0
4,2023-07-20 04:00:00+00:00,1167.9599609375,1169.6400146484375,1167.3199462890625,1168.239990234375,0


In [387]:

df.columns = ['thoi_gian', 'gia_dong_cua', 'gia_cao_nhat', 'gia_thap_nhat', 'gia_mo_cua', 'khoi_luong']

In [388]:
df['thoi_gian'] = pd.to_datetime(df['thoi_gian'], errors='coerce')
df['thoi_gian'] = df['thoi_gian'].dt.tz_convert('Asia/Ho_Chi_Minh')
df = df.dropna(subset=['thoi_gian']).reset_index(drop=True)

C:\Users\luan0\AppData\Local\Temp\ipykernel_5132\3384631275.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['thoi_gian'] = pd.to_datetime(df['thoi_gian'], errors='coerce')


In [389]:
df = df.drop(columns=['khoi_luong'])

In [390]:
cols = ['gia_dong_cua', 'gia_cao_nhat', 'gia_thap_nhat', 'gia_mo_cua',]
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

feature

bien dong

In [391]:
df['bien_do'] = df['gia_cao_nhat'] - df['gia_thap_nhat']
df['ty_le_thay_doi'] = (df['gia_dong_cua'] - df['gia_mo_cua']) / df['gia_mo_cua']

mo hinh nến

In [392]:
df['rau_tren'] = df['gia_cao_nhat'] - df[['gia_mo_cua', 'gia_dong_cua']].max(axis=1)
df['rau_duoi'] = df[['gia_mo_cua', 'gia_dong_cua']].min(axis=1) - df['gia_thap_nhat']
df['than_nen'] = abs(df['gia_dong_cua'] - df['gia_mo_cua'])
df['la_nen_xanh'] = (df['gia_dong_cua'] > df['gia_mo_cua']).astype(int)
df['ty_le_than_voi_range'] = df['than_nen'] / df['bien_do'].replace(0, np.nan)

xu hương

In [393]:
df['xu_huong_3'] = df['gia_dong_cua'] - df['gia_dong_cua'].shift(3)
df['xu_huong_5'] = df['gia_dong_cua'] - df['gia_dong_cua'].shift(5)

 Biến động ngắn hạn


In [394]:
df['do_lech_chuan_3'] = df['gia_dong_cua'].rolling(3).std()
df['do_lech_chuan_5'] = df['gia_dong_cua'].rolling(5).std()

cuốn chíu

In [395]:
df['trung_vi_5h'] = df['gia_dong_cua'].rolling(5).median()
df['max_5h'] = df['gia_dong_cua'].rolling(5).max()
df['min_5h'] = df['gia_dong_cua'].rolling(5).min()
df['do_lech_trung_vi'] = df['gia_dong_cua'] - df['trung_vi_5h']
df['skew_5h'] = df['gia_dong_cua'].rolling(5).skew()
df['kurt_5h'] = df['gia_dong_cua'].rolling(5).kurt()

khỏng trống giá

In [396]:
df['gap_len'] = ((df['gia_mo_cua'] - df['gia_dong_cua'].shift(1)) / df['gia_dong_cua'].shift(1)) > 0.01
df['gap_xuong'] = ((df['gia_mo_cua'] - df['gia_dong_cua'].shift(1)) / df['gia_dong_cua'].shift(1)) < -0.01

lag

In [397]:
for i in range(1, 11):
    df[f'gia_dong_cua_lag_{i}'] = df['gia_dong_cua'].shift(i)


In [398]:
df = df.dropna().reset_index(drop=True)

label

In [399]:
print(df.columns.tolist())


['thoi_gian', 'gia_dong_cua', 'gia_cao_nhat', 'gia_thap_nhat', 'gia_mo_cua', 'bien_do', 'ty_le_thay_doi', 'rau_tren', 'rau_duoi', 'than_nen', 'la_nen_xanh', 'ty_le_than_voi_range', 'xu_huong_3', 'xu_huong_5', 'do_lech_chuan_3', 'do_lech_chuan_5', 'trung_vi_5h', 'max_5h', 'min_5h', 'do_lech_trung_vi', 'skew_5h', 'kurt_5h', 'gap_len', 'gap_xuong', 'gia_dong_cua_lag_1', 'gia_dong_cua_lag_2', 'gia_dong_cua_lag_3', 'gia_dong_cua_lag_4', 'gia_dong_cua_lag_5', 'gia_dong_cua_lag_6', 'gia_dong_cua_lag_7', 'gia_dong_cua_lag_8', 'gia_dong_cua_lag_9', 'gia_dong_cua_lag_10']


In [400]:
n = 20
df['percent_change_future'] = (df['gia_dong_cua'].shift(-n) - df['gia_dong_cua']) / df['gia_dong_cua']
df['label'] = 0
df.loc[df['percent_change_future'] > 0.01, 'label'] = 2
df.loc[df['percent_change_future'] < -0.01, 'label'] = 1


In [401]:
print(df['label'].value_counts())


label
0    1027
2     904
1     549
Name: count, dtype: int64


model

In [402]:
X = df.drop(columns=['thoi_gian', 'label','percent_change_future'], errors='ignore')
y = df['label']
X = X.dropna()
y = y.loc[X.index]

In [403]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = xgb.XGBClassifier(
    objective='binary:logistic' if y.nunique() == 2 else 'multi:softmax',
    num_class=y.nunique() if y.nunique() > 2 else None,
    eval_metric='mlogloss' if y.nunique() > 2 else 'logloss',
    use_label_encoder=False,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))



c:\Users\luan0\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [18:58:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.56      0.64      0.60       205
           1       0.48      0.29      0.36       110
           2       0.64      0.69      0.67       181

    accuracy                           0.58       496
   macro avg       0.56      0.54      0.54       496
weighted avg       0.57      0.58      0.57       496



In [404]:
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)
print("Train label:", y_train.value_counts())
print("Test label:", y_test.value_counts())


Train shape: (1984, 33)
Test shape: (496, 33)
Train label: label
0    822
2    723
1    439
Name: count, dtype: int64
Test label: label
0    205
2    181
1    110
Name: count, dtype: int64


In [405]:
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
y_proba = model.predict_proba(X_test)

auc_score = roc_auc_score(y_test_bin, y_proba, multi_class='ovr')  

In [406]:
auc_score

np.float64(0.7714720877743874)